In [1]:
# !pip install 'zarr<3'
# !pip install timm
# !pip install openslide-python tiffslide
# !pip install cupy-cuda12x  # for CUDA 12.x

In [25]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## WSI Flex

In [26]:
import torch
from vitaminp import VitaminPFlex
from vitaminp.inference import ChannelConfig, WSIPredictor
# Setup model
device = 'cuda'
model = VitaminPFlex(model_size='large').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold2_best.pth", map_location=device))
model.eval()

✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]


VitaminPFlex(
  (backbone): DINOv2Backbone(
    (dinov2): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): M

## HE or mIF

In [28]:
from vitaminp.inference.channel_config import ChannelConfig
from vitaminp.inference import WSIPredictor

# Create channel config
config = ChannelConfig(
    nuclear_channel=0,  # SYTO 13
    membrane_channel=[1, 2],  # Combine all markers
    membrane_combination='max',
    channel_names={0: 'SYTO13', 1: 'Cy3', 2: 'TexasRed'}
)

# Create predictor with MIF config
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.4006,           # ← Default MPP (used if no metadata)
    magnification=40,          # ← Default magnification
    mif_channel_config=config
)

# Run MIF inference with MPP-aware filtering
results = predictor.predict(
    wsi_path='/rsrch9/home/plm/idso_fa1_pathology/TIER1/patient-mosaic/2014-0938/MOSAIC/DSP/MOSAIC DSP Set 1/Images/DSP Scans/MS001S1.ome.tiff',
    output_dir='results_new',
    branch='mif_nuclei',
    filter_tissue=False,
    # tissue_threshold=0.03,
    clean_overlaps=True,
    save_geojson=True,
    detection_threshold=0.5,      # ← Binary threshold (0.5-0.8)
    min_area_um=5.0,              # ← NEW: Biological size filter in μm²
    # mpp_override=0.5,           # ← Optional: Force specific MPP
)

print(f"✅ Found {results['num_detections']} nuclei in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")

# Check if any instances have area info
if results['num_detections'] > 0:
    first_cell = list(results['instances'].values())[0]
    if 'area_um' in first_cell:
        print(f"   First cell area: {first_cell['area_um']:.2f} μm² ({first_cell['area_pixels']:.0f} pixels²)")

2026-01-17 23:34:08 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-17 23:34:08 - WSIPredictor - INFO -   Device: cuda
2026-01-17 23:34:08 - WSIPredictor - INFO -   Model type: VitaminPFlex (single-modality)
2026-01-17 23:34:08 - WSIPredictor - INFO -   Patch size: 512
2026-01-17 23:34:08 - WSIPredictor - INFO -   Overlap: 64
2026-01-17 23:34:08 - WSIPredictor - INFO -   Magnification: 40
2026-01-17 23:34:08 - WSIPredictor - INFO -   MIF channels: Nuclear: SYTO13, Membrane: max(Cy3, TexasRed)
2026-01-17 23:34:08 - WSIPredictor - INFO -    Using default MPP: 0.4006 μm/px
2026-01-17 23:34:08 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-17 23:34:08 - WSIPredictor - INFO -    WSI MPP: 0.4006 μm/px
2026-01-17 23:34:08 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-17 23:34:08 - WSIPredictor - INFO -    Scale factor: 1.52x
2026-01-17 23:34:08 - WSIPredictor - INFO -    Min area filter: 5.0 μm² = 31 pixels²
2026-01-17 23:34:08 - WSIPredictor - INFO - 📁

Output: shape=(2, 32768, 32768), dtype=float32, range=[0.000, 1.000]


Processing tiles: 100%|██████████| 5329/5329 [18:22<00:00,  4.84it/s]
2026-01-17 23:54:35 - WSIPredictor - INFO -    ✓ Extracted 668914 instances from tiles (before cleaning)
2026-01-17 23:54:41 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 23:54:41 - WSIPredictor - INFO -    Found 177515 edge cells to check for overlaps
2026-01-17 23:55:04 - WSIPredictor - INFO - Starting overlap cleaning with 668914 detections
2026-01-17 23:55:04 - WSIPredictor - INFO - Split detections: 0 mid, 668914 edge
2026-01-17 23:56:41 - WSIPredictor - INFO - Iteration 1: Found 1908 overlaps, keeping 667006/668914 detections
2026-01-17 23:58:32 - WSIPredictor - INFO - Iteration 2: Found 0 overlaps, keeping 667006/667006 detections
2026-01-17 23:58:32 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 23:58:33 - WSIPredictor - INFO - Cleaning complete: 668914 -> 667006 (1908 removed)
2026-01-17 23:58:35 - WSIPredictor - INFO -    ✓ After cleanin

✅ Found 667006 nuclei in 1817.36s
   Output saved to: results_new


## Dual model

In [22]:
import torch
from vitaminp import VitaminPDual
from vitaminp.inference import ChannelConfig, WSIPredictor
# Setup model
device = 'cuda'
model = VitaminPDual(model_size='base').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_dual_base_fold1_best.pth", map_location=device))
model.eval()

Building H&E encoder with DINOv2-base
Building MIF encoder with DINOv2-base
Building shared encoder with DINOv2-base
✓ VitaminPDual initialized with base backbone
  Embed dim: 768 | Decoder dims: [768, 384, 192, 96]


VitaminPDual(
  (he_backbone): DINOv2Backbone(
    (dinov2): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(

In [24]:

# Corrected channel config for MIF
config = ChannelConfig(
    nuclear_channel=2,      # DAPI is in channel 2 (Blue)
    membrane_channel=[0, 1], # Cell markers in channels 0 and 1 (Red + Green)
    membrane_combination='max',
    channel_names={0: 'CellMarker1', 1: 'CellMarker2', 2: 'DAPI'}
)

# Default behavior (H&E predictions for H&E)
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.263,
    magnification=40,
    mif_channel_config=config
)

# 🔥 NEW: Use MIF predictions for H&E (seamlessly)
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.263,
    magnification=40,
    mif_channel_config=config,
)

# Same predict call - no changes needed!
results = predictor.predict(
    wsi_path='test_images/prostate-he_chunk_12.png',
    wsi_path_mif='test_images/prostate-mif_chunk_12_combined.png',
    output_dir='results_dual_new',
    branches=['he_nuclei', 'he_cell', 'mif_nuclei', 'mif_cell'],
    filter_tissue=True,
    clean_overlaps=True,
    save_geojson=True,
    save_visualization=True,
    detection_threshold=0.5,
    min_area_um=3.0,
)

# H&E results now contain MIF predictions automatically!
print(f"✅ H&E nuclei: {results['he_nuclei']['num_detections']}")
print(f"✅ H&E cells: {results['he_cell']['num_detections']}")

2026-01-17 22:44:06 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-17 22:44:06 - WSIPredictor - INFO -   Device: cuda
2026-01-17 22:44:06 - WSIPredictor - INFO -   Model type: VitaminPDual (dual-modality)
2026-01-17 22:44:06 - WSIPredictor - INFO -   Patch size: 512
2026-01-17 22:44:06 - WSIPredictor - INFO -   Overlap: 64
2026-01-17 22:44:06 - WSIPredictor - INFO -   Magnification: 40
2026-01-17 22:44:06 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: max(CellMarker1, CellMarker2)
2026-01-17 22:44:06 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-17 22:44:06 - WSIPredictor - INFO -   Device: cuda
2026-01-17 22:44:06 - WSIPredictor - INFO -   Model type: VitaminPDual (dual-modality)
2026-01-17 22:44:06 - WSIPredictor - INFO -   Patch size: 512
2026-01-17 22:44:06 - WSIPredictor - INFO -   Overlap: 64
2026-01-17 22:44:06 - WSIPredictor - INFO -   Magnification: 40
2026-01-17 22:44:06 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: 

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ Extracted 91 instances from tiles (before cleaning)
2026-01-17 22:44:07 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:07 - WSIPredictor - INFO -    Found 33 edge cells to check for overlaps
2026-01-17 22:44:07 - WSIPredictor - INFO - Starting overlap cleaning with 91 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - Split detections: 0 mid, 91 edge
2026-01-17 22:44:07 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 91/91 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:07 - WSIPredictor - INFO - Cleaning complete: 91 -> 91 (0 removed)
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ After cleaning: 91 instances
2026-01-17 22:44:07 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/he_nuclei...
2026-01-17 22:44:07 - WSIPredictor - INFO 

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  4.49it/s]
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ Extracted 89 instances from tiles (before cleaning)
2026-01-17 22:44:07 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:07 - WSIPredictor - INFO -    Found 34 edge cells to check for overlaps
2026-01-17 22:44:07 - WSIPredictor - INFO - Starting overlap cleaning with 89 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - Split detections: 0 mid, 89 edge
2026-01-17 22:44:07 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 89/89 detections
2026-01-17 22:44:07 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:07 - WSIPredictor - INFO - Cleaning complete: 89 -> 89 (0 removed)
2026-01-17 22:44:07 - WSIPredictor - INFO -    ✓ After cleaning: 89 instances
2026-01-17 22:44:07 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/he_cell...
2026-01-17 22:44:07 - WSIPredictor - INFO - 

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  4.74it/s]
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ Extracted 91 instances from tiles (before cleaning)
2026-01-17 22:44:08 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:08 - WSIPredictor - INFO -    Found 33 edge cells to check for overlaps
2026-01-17 22:44:08 - WSIPredictor - INFO - Starting overlap cleaning with 91 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - Split detections: 0 mid, 91 edge
2026-01-17 22:44:08 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 91/91 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:08 - WSIPredictor - INFO - Cleaning complete: 91 -> 91 (0 removed)
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ After cleaning: 91 instances
2026-01-17 22:44:08 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/mif_nuclei...
2026-01-17 22:44:08 - WSIPredictor - INFO

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  3.96it/s]
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ Extracted 89 instances from tiles (before cleaning)
2026-01-17 22:44:08 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-17 22:44:08 - WSIPredictor - INFO -    Found 34 edge cells to check for overlaps
2026-01-17 22:44:08 - WSIPredictor - INFO - Starting overlap cleaning with 89 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - Split detections: 0 mid, 89 edge
2026-01-17 22:44:08 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 89/89 detections
2026-01-17 22:44:08 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-17 22:44:08 - WSIPredictor - INFO - Cleaning complete: 89 -> 89 (0 removed)
2026-01-17 22:44:08 - WSIPredictor - INFO -    ✓ After cleaning: 89 instances
2026-01-17 22:44:08 - WSIPredictor - INFO - 💾 Saving results to results_dual_new/mif_cell...
2026-01-17 22:44:08 - WSIPredictor - INFO -

Output: shape=(2, 512, 512), dtype=float32, range=[0.000, 1.000]
✅ H&E nuclei: 91
✅ H&E cells: 89


## DSP set 1

In [ ]:
from vitaminp.inference.channel_config import ChannelConfig
from vitaminp.inference import WSIPredictor

# Create channel config
config = ChannelConfig(
    nuclear_channel=0,  # SYTO 13
    membrane_channel=[1, 2],  # Combine all markers
    membrane_combination='max',
    channel_names={0: 'SYTO13', 1: 'Cy3', 2: 'TexasRed'}
)

# Create predictor with MIF config
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.5013,           # ← Default MPP (used if no metadata)
    magnification=20,          # ← Default magnification
    mif_channel_config=config
)

# Run MIF inference with MPP-aware filtering
results = predictor.predict(
    wsi_path='test_images/MS004_HE.svs',
    output_dir='results_new',
    branch='he_nuclei',
    filter_tissue=False,
    # tissue_threshold=0.03,
    clean_overlaps=True,
    save_geojson=True,
    detection_threshold=0.5,      # ← Binary threshold (0.5-0.8)
    min_area_um=5.0,              # ← NEW: Biological size filter in μm²
    # mpp_override=0.5,           # ← Optional: Force specific MPP
)

print(f"✅ Found {results['num_detections']} nuclei in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")

# Check if any instances have area info
if results['num_detections'] > 0:
    first_cell = list(results['instances'].values())[0]
    if 'area_um' in first_cell:
        print(f"   First cell area: {first_cell['area_um']:.2f} μm² ({first_cell['area_pixels']:.0f} pixels²)")